In [2]:
# import ragas.metrics as m
import os
from dotenv import load_dotenv
import pandas as pd
from src.extractor import relationExtractor
from src.llms import OpenAIModel, HuggingFaceLLM
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv()
link = os.getenv('dsa_2214')
token = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN') 
os.environ['OPENAI_API_KEY'] = token

chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

In [8]:
# data = pd.read_csv('data/sorting.csv')
# data.columns = ['concept', 'outcome']
data = pd.read_csv('data/dsa_clifford_a_shaffer_3_2_java.csv')
data

,chapter,concepts,outcomes
0,Data Structures and Algorithms,Data Structures::Algorithms::Algorithm Efficie...,Demonstrate an understanding of what algorthms...
1,Mathematical Preliminaries,Sets and Relations::Set Notation::Boolean logi...,Demonstrate an understanding of sets::Demonstr...
2,Algorithm Analysis,Asymptotic Analysis::Big O notation::Algorithm...,Demonstrate an understanding of algorithm anal...
3,Lists Stacks and Queues,Lists::Stacks::Queues::Array-based List Implem...,Demonstrate an understanding of Lists::Demonst...
4,Binary Trees,Nodes::Binary Search Tree::Binary Tree::Subtre...,Demonstrate an understanding of binary trees::...
5,Non-Binary Trees,General Tree Traversals::General Tree Node Abs...,Demonstrate an understanding of trees::Demonst...
6,Internal Sorting,Mergesort::Quicksort::Shellsort::Heapsort::Sor...,Demonstrate an understanding of sorting algori...
7,File Processing and External Sorting,Primary Memory::Secondary Storage::Random Acce...,Demonstrate an understanding of disk drives::D...
8,Searching,Searching::Search Queries::Linear Search::Jump...,Demonstrate an understanding of search algorit...
9,Indexing,Indexing::Primary Keys::Secondary Keys::Tree I...,Demonstrate an understanding of primary and se...


In [ ]:
concept_data = data['concept'].tolist()
actual_concepts = []
for string in concept_data:
    words = string.split('->')
    for word in words:
        if word not in actual_concepts:
            actual_concepts.append(word)


# outcome_data = data['outcome'].tolist()
# actual_outcomes = []
# for string in outcome_data:
#     words = string.split(';')
#     for s in words:
#         if s not in actual_outcomes:
#             actual_outcomes.append(s)

actual_concepts = [' '.join(actual_concepts)] * 1
# actual_outcomes = [' '.join(actual_outcomes)] * 4

In [2]:
extractor = relationExtractor(link,
                            [chapters[3]], 
                            300, 
                            100, 
                            HuggingFaceLLM('LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct'))

The repository for LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 70.00 MiB. GPU 

In [ ]:
from src.metrics import SemanticSimilarity, Contextual_F1
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric, FaithfulnessMetric, ContextualRelevancyMetric


metrics = [AnswerRelevancyMetric(model = extractor.llm), FaithfulnessMetric(model = extractor.llm), ContextualPrecisionMetric(model = extractor.llm), ContextualRecallMetric(model = extractor.llm), SemanticSimilarity(st_model = extractor.embedding_model)]

In [ ]:
# for i in range(5):
result = extractor.evaluate('concepts', 10, actual_concepts, metrics)
print('-' * 20)
print(f'OPENAI MODEL: {extractor.llm.get_model_name()}')
print(f'TEMPERATURE: {extractor.llm.temperature}')
print(f'SENTENCE TRANSFORMER: {extractor.embedding_model}')
print(f'TEXTBOOK: dsa_2214')
print(f'CHAPTERS TESTED: {extractor.chapters}')
print('-' * 20)
for i in result:
    print(f"METRIC: {i['name']} ----> SCORE: {i['score']}")
    print()
    print(f"REASON: {i['reason']}")
    print() 
    print(f"QUERY: {i['input']}")
    print()
    print(f"OUTPUT: {i['output']}")
    print('-' * 20)

In [ ]:
full_extractor = relationExtractor(link, 
                                chapters, 
                                500, 
                                100, 
                                OpenAIModel())

In [ ]:
full_extractor.identify_concepts(5)
full_extractor.identify_key_terms(5)
full_extractor.identify_outcomes(5)

In [ ]:
full_extractor.identify_dependencies(full_extractor.key_terms) 

In [ ]:
full_extractor.build_knowledge_graph()

In [ ]:
full_extractor.draw_knowledge_graph()

In [ ]:
full_extractor.build_terminology()

In [ ]:
full_extractor.draw_hierarchy()

In [ ]:
full_extractor.terminology

In [ ]:
from src.retrieval import RetrievalSystem
from langchain_openai import ChatOpenAI

In [ ]:
rs = RetrievalSystem(link, 700, 50, 'msmarco-distilbert-base-tas-b')

In [ ]:
results = rs.pipeline('What is the limits of computation chapter about?', ChatOpenAI(), 4, 10)
results

In [ ]:
ChatOpenAI(model = 'gpt-4o-mini', temperature = 0).invoke(f"Identify 5 key terms from chapter Limits to Computation. The textbook is here: {link}")

In [ ]:
rs.pipeline('Turing Machine, Decidability, Halting Problem, Intractability', ChatOpenAI())

In [ ]:
response = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0).invoke('Provide 5 questions for the topics of Limits of Computation. These should be relatively simple questions. Provide your response as a list of questions.')
response.content

In [ ]:
rs.pipeline('Lists, Stacks, and Queues', ChatOpenAI(model = 'gpt-4o'))

## 